In [1]:
import pandas as pd
import subprocess
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn import metrics
import scipy.stats as sp
import sklearn.metrics

# Part 1: Read in both predictions and ground truth

In [2]:
preds = pd.read_csv('../out/hepg2_toxpreds_on_training_plus_ml_curation_hits.csv')
preds['Predicted HepG2'] = preds['TOXICITY']
preds = preds[['SMILES', 'Name', 'hit_inh', 'hit_kill', 'Predicted HepG2']]
preds

,SMILES,Name,hit_inh,hit_kill,Predicted HepG2
0,CCCC[C@H](CC)CNC(=N)N=C(N)NCCCCCCNC(N)=NC(=N)N...,alexidine,1.0,1.0,0.263691
1,NC(Nc1ccc(Cl)cc1)=NC(=N)NCCCCCCNC(=N)N=C(N)Nc1...,chlorhexidine,1.0,1.0,0.684506
2,CC[C@@H](C)CCCCC(=O)N[C@@H](CCN)C(=O)N[C@@H]([...,polymyxin-B-sulfate,1.0,1.0,0.058804
3,OCCN[C@H]1CCCc2c1[nH]c1ccc(cc21)-c1ccccc1 |&1:...,casin,1.0,1.0,0.416008
4,NC[C@H]1O[C@H](O[C@@H]2[C@@H](N)C[C@@H](N)[C@H...,bekanamycin,1.0,1.0,0.026443
...,...,...,...,...,...
95,CC(=O)O[Pt+4](C)(C)(Cl)(Cl)OC(=O)C.NC1CCCCC1,BRD-M31395033,0.0,1.0,0.177704
96,CN(C)c1ccc(cc1)[C+](c1ccc(cc1)N(C)C)c1ccc(cc1)...,BRD-K60025295,0.0,1.0,0.690095
97,CN1CCN(CC1)c1c(cc2c3c1OCN(C)N3C=C(C(=O)O)C2=O)F,BRD-K71926323,1.0,1.0,0.037562
98,CC(C)c1ccc(cc1)Cn1ccc2c1ccc1nc(nc(c12)N)NC1CC1,BRD-K17140735,1.0,1.0,0.588827


In [3]:
groundtruth = pd.read_excel('../data/Fig 3C cytotoxicity.xlsx')
groundtruth['HepG2'] = [np.mean([x,y]) for x,y in zip(list(groundtruth.iloc[:,1]), list(groundtruth.iloc[:,2]))]
groundtruth['HEK293'] = [np.mean([x,y]) for x,y in zip(list(groundtruth.iloc[:,3]), list(groundtruth.iloc[:,4]))]
groundtruth = groundtruth[['Unnamed: 0', 'HepG2', 'HEK293']]
groundtruth.columns = ['Broad_ID', 'HepG2', 'HEK293']
groundtruth

,Broad_ID,HepG2,HEK293
0,BRD-K18552138-001-01-8,1.261710,1.016640
1,BRD-K09471561-001-18-2,0.865476,0.879784
2,BRD-K90991629-100-02-9,0.900257,0.922629
3,BRD-K00494077-066-02-7,1.038782,0.855533
4,BRD-K01825499-003-03-9,1.079024,0.987306
...,...,...,...
95,BRD-K00003300-001-01-9,1.345007,1.013332
96,BRD-K00003240-300-01-9,1.349115,0.922583
97,BRD-K01666924-319-01-9,1.350019,1.050155
98,BRD-K41731458-001-15-1,1.354185,0.993971


In [4]:
# get predictions along with their Broad IDs to merge on
mapped_names = pd.read_excel('../data/TrainingDataRound1_wValidation.xlsx')
mapped_names = mapped_names[['Sample', 'Name']]
mapped_names.columns = ['Broad_ID', 'Name']

primscreen = preds.merge(mapped_names, on = 'Name', how = 'inner')
mlcuration = preds[['BRD' in x for x in list(preds['Name'])]]
mlcuration['Broad_ID'] = mlcuration['Name']
preds = pd.concat([primscreen,mlcuration])
preds

/var/folders/py/8hwb28r933z4xh4q0dw5hz2h0000gn/T/ipykernel_23561/107771575.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlcuration['Broad_ID'] = mlcuration['Name']


,SMILES,Name,hit_inh,hit_kill,Predicted HepG2,Broad_ID
0,CCCC[C@H](CC)CNC(=N)N=C(N)NCCCCCCNC(N)=NC(=N)N...,alexidine,1.0,1.0,0.263691,BRD-A42335949-300-09-8
1,NC(Nc1ccc(Cl)cc1)=NC(=N)NCCCCCCNC(=N)N=C(N)Nc1...,chlorhexidine,1.0,1.0,0.684506,BRD-K52256627-300-05-8
2,CC[C@@H](C)CCCCC(=O)N[C@@H](CCN)C(=O)N[C@@H]([...,polymyxin-B-sulfate,1.0,1.0,0.058804,BRD-A08641215-065-05-5
3,OCCN[C@H]1CCCc2c1[nH]c1ccc(cc21)-c1ccccc1 |&1:...,casin,1.0,1.0,0.416008,BRD-A25043798-001-02-9
4,NC[C@H]1O[C@H](O[C@@H]2[C@@H](N)C[C@@H](N)[C@H...,bekanamycin,1.0,1.0,0.026443,BRD-K73425385-001-05-9
...,...,...,...,...,...,...
95,CC(=O)O[Pt+4](C)(C)(Cl)(Cl)OC(=O)C.NC1CCCCC1,BRD-M31395033,0.0,1.0,0.177704,BRD-M31395033
96,CN(C)c1ccc(cc1)[C+](c1ccc(cc1)N(C)C)c1ccc(cc1)...,BRD-K60025295,0.0,1.0,0.690095,BRD-K60025295
97,CN1CCN(CC1)c1c(cc2c3c1OCN(C)N3C=C(C(=O)O)C2=O)F,BRD-K71926323,1.0,1.0,0.037562,BRD-K71926323
98,CC(C)c1ccc(cc1)Cn1ccc2c1ccc1nc(nc(c12)N)NC1CC1,BRD-K17140735,1.0,1.0,0.588827,BRD-K17140735


In [5]:
# clean names to standardize
preds['Broad_ID'] = [x.split('-')[0] + '-' + x.split('-')[1] for x in list(preds['Broad_ID'])]
groundtruth['Broad_ID'] = [x.split('-')[0] + '-' + x.split('-')[1] for x in list(groundtruth['Broad_ID'])]

# merged tox dataframe with predictions and groundtruth
toxdf = preds.merge(groundtruth, on = 'Broad_ID')
toxdf

,SMILES,Name,hit_inh,hit_kill,Predicted HepG2,Broad_ID,HepG2,HEK293
0,CCCC[C@H](CC)CNC(=N)N=C(N)NCCCCCCNC(N)=NC(=N)N...,alexidine,1.0,1.0,0.263691,BRD-A42335949,0.102992,0.201796
1,NC(Nc1ccc(Cl)cc1)=NC(=N)NCCCCCCNC(=N)N=C(N)Nc1...,chlorhexidine,1.0,1.0,0.684506,BRD-K52256627,0.412102,0.396537
2,CC[C@@H](C)CCCCC(=O)N[C@@H](CCN)C(=O)N[C@@H]([...,polymyxin-B-sulfate,1.0,1.0,0.058804,BRD-A08641215,1.085333,0.919343
3,OCCN[C@H]1CCCc2c1[nH]c1ccc(cc21)-c1ccccc1 |&1:...,casin,1.0,1.0,0.416008,BRD-A25043798,0.087166,0.154540
4,NC[C@H]1O[C@H](O[C@@H]2[C@@H](N)C[C@@H](N)[C@H...,bekanamycin,1.0,1.0,0.026443,BRD-K73425385,1.138278,0.906419
...,...,...,...,...,...,...,...,...
95,CC(=O)O[Pt+4](C)(C)(Cl)(Cl)OC(=O)C.NC1CCCCC1,BRD-M31395033,0.0,1.0,0.177704,BRD-M31395033,0.960815,0.626337
96,CN(C)c1ccc(cc1)[C+](c1ccc(cc1)N(C)C)c1ccc(cc1)...,BRD-K60025295,0.0,1.0,0.690095,BRD-K60025295,0.048104,0.885436
97,CN1CCN(CC1)c1c(cc2c3c1OCN(C)N3C=C(C(=O)O)C2=O)F,BRD-K71926323,1.0,1.0,0.037562,BRD-K71926323,0.904802,0.790365
98,CC(C)c1ccc(cc1)Cn1ccc2c1ccc1nc(nc(c12)N)NC1CC1,BRD-K17140735,1.0,1.0,0.588827,BRD-K17140735,0.038520,0.067379


# Part 2: Evaluate predictions with experimental data

In [6]:
# evaluate each prediction
def modeleval(y_true, y_pred, name = ''):
    
    new_ytrue = []
    new_ypred = []
    for x, y in zip(y_true, y_pred):
        if y != 'Invalid SMILES':
            try:
                new_ytrue.append(int(x))
                new_ypred.append(float(y))
            except Exception as e:
                #print(x)
                #print(e)
                continue
            
    y_true = new_ytrue
    y_pred = new_ypred

    auroc = float(roc_auc_score(y_true, y_pred))
    print('auroc: ' + str(auroc))

    # Compute Precision-Recall and plot curve
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr = float(auc(recall,precision))
    print('precision recall: ' + str(pr))

    return(auroc, pr, y_true, y_pred)

In [7]:
thresh = 0.2

# first compare hepg2 preds with hepg2 actual
groundtruthvals = list(toxdf['HepG2'])
groundtruthbinaryvals = [0 if x > thresh else 1.0 for x in groundtruthvals] # because its growth not toxicity, so the lower the growth the more toxic

pred_vals = list(toxdf['Predicted HepG2'])
auroc, pr, y_true, y_pred = modeleval(groundtruthbinaryvals, pred_vals)
sp.pearsonr(y_true, y_pred)

# and binary class too
tests = [1.0 if x else 0.0 for x in [y > thresh for y in y_pred]]
    
print('num positives: ', sum(tests))
print('mcc: ', sklearn.metrics.matthews_corrcoef(y_true, tests))
print('spearman r: ', sp.spearmanr(y_true, tests))
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_true, tests).ravel()
print(tn,fp,fn,tp)

# fnr = fn/fn+tp = 3/(3+29) = 9.4% = low!

auroc: 0.6847426470588236
precision recall: 0.39562668736602
num positives:  63.0
mcc:  0.3925108649002968
spearman r:  SpearmanrResult(correlation=0.3925108649002968, pvalue=5.365612861913739e-05)
34 34 3 29


In [8]:
# now compare hepg2 preds with hek23 actual
groundtruthvals = list(toxdf['HEK293'])
groundtruthbinaryvals = [0 if x > thresh else 1.0 for x in groundtruthvals] # because its growth not toxicity, so the lower the growth the more toxic

pred_vals = list(toxdf['Predicted HepG2'])
auroc, pr, y_true, y_pred = modeleval(groundtruthbinaryvals, pred_vals)
sp.pearsonr(y_true, y_pred)

# and binary class too
tests = [1.0 if x else 0.0 for x in [y > thresh for y in y_pred]]
    
print('num positives: ', sum(tests))
print('mcc: ', sklearn.metrics.matthews_corrcoef(y_true, tests))
print('spearman r: ', sp.spearmanr(y_true, tests))
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_true, tests).ravel()
print(tn,fp,fn,tp)
# fnr = fn/fn+tp = 4/(4+24) = 14.3% = low!

auroc: 0.7137896825396826
precision recall: 0.4524495950848284
num positives:  63.0
mcc:  0.2933868977820201
spearman r:  SpearmanrResult(correlation=0.2933868977820201, pvalue=0.0030518535311729648)
33 39 4 24
